# Db2 Macros
Update: 2019-10-03

The `%sql` command also allows the use of macros. Macros are used to substitute text into SQL commands that you execute. Macros substitution is done before any SQL is executed. This allows you to create macros that include commonly used SQL commands rather than having to type them in. Before using any macros, we must make sure we have loaded the Db2 extensions.

In [ ]:
%run ../db2.ipynb
%run ../connection.ipynb

### Macro Basics
A Macro command contains logic and SQL text that you want to have substituted into your script. The macro name is the first keyword after the `%sql` command (ignoring any flags). For instance, the following `%sql` command would consider `LIST` as the macro name:
```
%sql LIST TABLES FOR SCHEMA DB2INST1
```
The `%sql` processor first checks to see if this first keyword has been defined as a macro. If not, it will process it as a regular SQL statement. 

To define a macro, the `%%sql define <name>` command is used. The body of the macro is found in the cell below the definition of the macro. This simple macro called `EMPTABLE` will substitute a `SELECT` statement into a SQL block.

In [ ]:
%%sql define emptable
select * from employee 

The name of the macro follows the `%%sql define` command and is **not** case sensitive. To use the macro, we can place it anywhere in a `%sql` block. This first example uses the macro by itself.

In [ ]:
%%sql
emptable

The actual SQL that is generated is not shown by default. If you do want to see the SQL that gets generated, you can use the `-e` (echo) option to display the final SQL statement. The following example will display the generated SQL. Note that the echo setting is only used to display results for the current cell that is executing.

In [ ]:
%%sql -e
emptable

You can use the `emptable` macro anywhere in a SQL block but care must be taken to terminate the macro with a semi-colon to delimit it from other statements in the block. SQL requires that statements be separated with semi-colons and a macro is treated as a SQL statement. Any text following the name of the macro up to a semi-colon (or the end of the block) will be treated as belonging to the macro. 

In this example we add some logic after the `emptable` macro.

In [ ]:
%%sql -e
EMPTABLE lastname
where empno = '000010';

Note that the logic after `SELECT` statement was completely ignored. All of the text following the macro, **up to a semi-colon or end of the block**, is considered part of the macro. There was no code in the macro body that indicated what to do with all of the tokens.

The text that follows the name of the macro is tokenized (split by spaces). In order to use any tokens that follow the macro name, you can refer to them in your macro body using the special sequence `{#}` where the number sign represents the nth token in the string. The tokens that are placed after the name of the macro can be any number, character string, or quoted string that is separated by blanks. For instance, the following string has 5 tokens:
```
LIST 1234 "Here's a string" AbCD-12 'String'
```
The tokens are `LIST`, `1234`, `"Here's a string"`, `AbCD-12`, and `'String'`. Note that quotes will ensure that blanks inside the string will not split the string into multiple tokens. Tokens are numbered from 0 to n-1 where token number 0 (zero) is the name of the macro and 1 to n-1 are the tokens following the macro name. In the example above, token 0 is "LIST" while 1 is 1234 and so on.

Note that the parsing is relatively simple. If you have a string similar to the following, it will not observe SQL parsing rules like the use of parenthesis and brackets.
```
LIST ("this is","parameter") FROM EMPLOYEE
```
The parser will generate the following tokens.
```
'LIST', '("this is","parameter")', 'FROM', 'EMPLOYEE'
```
From a macro perspective this means that any tokens that you want to pass to the routine needs to be kept relatively simple and separated with blanks.

Here is a simple macro which will use the first token as the name of the column we want returned from the EMPLOYEE table. Note that token 0 is the name of the macro and the remainder of the parameters are numbered from 1 to the total number of values.

In [ ]:
%%sql define emptable
SELECT {1} FROM EMPLOYEE

This example illustrates two concepts. The `DEFINE` command will replace any existing macro with the same name. Since we already have an `EMPTABLE` macro, the macro body will be replaced with this code. In addition, macros only exist within your notebook. If you create another Jupyter notebook, it will not contain any macros that you may have created. If there are macros that you want to share across notebooks, you should create a separate notebook and place all of the macro definitions in there. Then you can include these macros by executing the `%run` command using the name of the notebook that contains the macros.

The following SQL shows the use of the macro with parameters.

In [ ]:
%%sql
emptable lastname

Back to the example with the logic following the macro name. How can any tokens after a macro be referred to in the body of the macro? As previously shown, to refer to an individual token you would use the `{#}` syntax. If you don't know how many tokens there are, you can refer to everything **including** the current token with the syntax `{*#}`. This will generate a concatenated string of all tokens starting with token number `#`, each separated with a blank space.

The following macro definition will now place all of the text after the macro name in the generated SQL.

In [ ]:
%%sql define emptable
SELECT * FROM EMPLOYEE {*1}

Now our previous example will work properly.

In [ ]:
%%sql -e
emptable where empno = '000010'

Since a SQL statement is delimited by a semi-colon, or by the end of the block, the logic can be placed on a separate line.

In [ ]:
%%sql -e
emptable
where empno = '000010'

The remainder of this notebook will explore the advanced features of macros.

## Macro Tokens
Macros can have any number of tokens associated with them. The tokens are numbered from 1 to n, left to right as they are found in the statement. Tokens are delimited by blanks and quotes strings will have blanks ignored. The following macro has 5 tokens:
```
emptable lastname firstnme salary bonus '000010'
```
Tokens are separated by spaces, and can contain strings as shown using single or double quotes. When the tokens are used within a macro, the quotes **are included as part of the string**. 

In [ ]:
%%sql define emptable
SELECT {1},{2},{3},{4} 
FROM EMPLOYEE
WHERE EMPNO = {5}

Note that the `EMPNO` field is a character field in the `EMPLOYEE` table so you need to decide whether or not the user supplies the value in quotes (`'000010'`) or you add them to the token. This first example assumes that you supply the quotes as part of the macro.

In [ ]:
%sql -e emptable lastname firstnme salary bonus '000010'

We can modify the macro to assume that the parameters will not include the quotes in the string.

In [ ]:
%%sql define emptable
SELECT {1},{2},{3},{4} 
FROM EMPLOYEE
WHERE EMPNO = '{5}'

We just have to make sure that no quotes are part of the parameter now.

In [ ]:
%sql -e emptable lastname firstnme salary bonus 000010

If you just want all tokens to be included in the generated SQL, you can use the syntax `{*#}`. This will generate a concatenated string of all tokens starting with token number `#`, each separated with a blank space. The next macro will just select everything from the `EMPLOYEE` table by default and then append all of the remaining tokens after the select statement.

In [ ]:
%%sql define emptable
SELECT * FROM EMPLOYEE
{*1}

Now the `EMPTABLE` macro will select all of the columns from the table and add the additional logic that was supplied after the macro name.

In [ ]:
%%sql
emptable
where empno = '000010'

## Macro Coding Overview
Macros can contain any type of text, including SQL commands. In addition to the text, macros can also contain the following keywords:

* \# - Comment (first character in the line)
* echo - Display a message
* return - Exit the macro but return any SQL that was generated
* exit - Exit the macro immediately with no generated code
* if/else/endif - Conditional logic
* var - Set a variable

The only restriction with macros is that macros cannot be nested. This means I can't call a macro from within a macro. The sections below explain the use of each of these statement types.

### Echo Option
The `-e` option will result in the final SQL being display after the macro substitution is done. 
```
%%sql -e
%showemp(...)
```

In [ ]:
%%sql define showdisplay
SELECT * FROM EMPLOYEE FETCH FIRST ROW ONLY

Using the `-e` flag will display the final SQL that is run.

In [ ]:
%sql -e showdisplay

If we remove the `-e` option, the final SQL will not be shown.

In [ ]:
%sql showdisplay

### Comment
You can comment out lines or add documentation to your macros by placing a `#` symbol at the beginning of any line. The `#` symbol will only be recognized as a comment if there is nothing in front of it, otherwise it is considered part of the SQL you are generating.

The following is an example of a macro definition with some comments.

In [ ]:
%%sql define hello
#
# This macro will display a Hello World Message
#
echo Hello World

In [ ]:
%sql hello

### Exit Command and Return Command
The `exit` command will terminate the processing within a macro and not run the generated SQL. You would use this when a condition is not met within the macro (like a missing parameter). The `exit` command can contain a message that will be displayed before terminating the macro. 

The `return` command will also stop the processing of the macro but will return any generated SQL back for execution.

In [ ]:
%%sql define showexit
echo This message gets shown
SELECT * FROM EMPLOYEE FETCH FIRST ROW ONLY
exit I just exited the macro
echo This message does not get shown

The macro that was defined will not show the second statement, nor will it execute the SQL that was defined in the macro body. Note that the echo command displays messages in green while the exit command displays it in red.

In [ ]:
%sql showexit

If we change the `exit` to a `return`, the SQL will be executed.

In [ ]:
%%sql define showexit
echo This message gets shown
SELECT * FROM EMPLOYEE FETCH FIRST ROW ONLY
return
echo This message does not get shown

In [ ]:
%sql showexit

### Echo Command
As you already noticed in the previous example, the `echo` command will display information on the screen. Any text following the command will have variables substituted and then displayed with a green box surrounding it. The following code illustrates the use of the command.

In [ ]:
%%sql define showecho
echo Here is a message
echo Two lines are shown

The echo command will show each line as a separate box.

In [ ]:
%sql showecho

If you want to have a message go across multiple lines use the `<br>` to start a new line.

In [ ]:
%%sql define showecho
echo Here is a paragraph. <br>And a final paragraph.

In [ ]:
%sql showecho

### Var Command
The var (variable) command sets a macro variable to a value. A variable is referred to in the macro script using curly braces `{name}`. By default the arguments that are used in the macro call are assigned the variable names `{0}` to `{n-1}`. The variable zero contains the name of the macro, while each token after the macro name is assigned to one of these numbered variables. There is also a system variable `{argc}` which contains a count of the number of tokens found (including the name of the macro). 

To set a variable within a macro you would use the `var` command:
```
var name value
```
The variable name can be any name as long as it only includes letters, numbers, underscore `_` and `$`. Variable names are case sensitive so `{a}` and `{A}` are different. When the macro finishes executing, the contents of the variables will be lost. 

A variable can be converted to uppercase by placing the `^` beside the variable name or number. 

In [ ]:
%%sql define runit
echo The first parameter is {^1}

In [ ]:
%sql runit Hello There

The string following the variable name can include quotes and these will not be removed. 

In [ ]:
%%sql define runit
var hello This is a long string without quotes
var hello2 'This is a long string with quotes'
echo {hello} <br>{hello2}

In [ ]:
%sql runit

When processing a macro, each one of the tokens is automatically assigned to a variable (1, 2, ..., n) and variable zero `{0}` is assigned the name of the macro. The following macro will be used to show how all of the tokens are passed to the routine.

In [ ]:
%%sql define showvar
echo Token(1)={1} <br>Token(2)={2} <br>All={*0}<br>After Token 0={*1}<br>Count={argc}

Calling the macro will show how the variable names get assigned and used.

In [ ]:
%sql showvar hello there everyone

If the token does not exist, a null keyword will be shown. If you use `{*#}` then if no values are found an empty string is returned.

In [ ]:
%sql showvar hello

Finally, any string that is supplied to the macro will include the quotes in the variable. The Hello There string will include the quotes when displayed:

In [ ]:
%sql showvar "Hello There" Everyone

The count of the total number of parameters passed is found in the `{argc}` variable. You can use this variable to decide whether or not the user has supplied the proper number of tokens or change which code should be executed.

In [ ]:
%%sql define showvar
echo The number of unnamed parameters is {argc}. 

The count of tokens does not include the name of the macro. 

In [ ]:
%sql showvar Hello There

### If/Else/Endif Command
If you need to add conditional logic to your macro then you should use the `if/else/endif` commands. The format of the `if` statement is:
```
if variable condition value
   statements
else
   statements
endif
```
The else portion is optional, but the block must be closed with the `endif` command. If statements can be nested up to 9 levels deep:
```
if condition 1
   if condition 2
      statements
   else
      if condition 3
         statements
      end if 
   endif
endif
```
If the condition in the if clause is true, then anything following the if statement will be executed and included in the final SQL statement. For instance, the following code will create a SQL statement based on the value of parameter 1:
```
if {1} = ""
   SELECT * FROM EMPLOYEE
else
   SELECT {1} FROM EMPLOYEE
endif
```

#### Conditions
The `if` statement requires a condition to determine whether or not the block should be executed. The condition uses the following format:
```
if {variable} condition {variable} | constant | null
```
`Variable` can be a number from 1 to n which represents the tokens that are parsed during the macro call. So `{1}` refers to the first argument. The variable can also be the name of a named parameter.

The condition is one of the following comparison operators:
- `=`, `==`: Equal to
- `<`: Less than
- `>`: Greater than
- `<=`,`=<`: Less than or equal to
- `>=`, `=>`: Greater than or equal to
- `!=`, `<>` : Not equal to

The variable or constant will have quotes stripped away before doing the comparison. If you are testing for the existence of a variable, or to check if a token is empty, use the `null` keyword.

In [ ]:
%%sql define showif
if {argc} = 0
   exit No parameters supplied
else
   if {argc} = "1"
       echo One parameter was supplied
   else
       echo More than one parameter was supplied: {argc}
   endif
endif

Running the previous macro with no parameters will check to see if the option keyword was used.

In [ ]:
%sql showif

Now include one parameter.

In [ ]:
%sql showif "Here is a token"

Finally, issue the macro with multiple token.

In [ ]:
%sql showif Here are a number of parameters

One additional option is available for variable substitution. If the first character of the variable name or parameter number is the `^` symbol, it will uppercase the entire string.

In [ ]:
%%sql define showif
if {1} <> null
   echo The first token is: {^1}
else
   echo You didn't supply a token
endif

In [ ]:
%sql showif "Yes there is an option"

In [ ]:
%sql showif

#### Credits: IBM 2019, George Baklarz [baklarz@ca.ibm.com]